In [1]:
import os
import json
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
#The Goal: 
# A supervised learning algorithm, such as a decision tree or support vector machine, 
# could be used to classify the genre of music based on the data provided. 
# The algorithm would be trained on a dataset of labeled music genres and the associated data values, 
# and then it could be used to predict the genre of music given a new set of data values.

cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='https://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [3]:
#Creating function and having user give PlaylistID, and genre (0 = counrty, 1 = rock...)
def input_playlist(playlistID, genre): 

    playlist = sp.user_playlist(username, playlistID);

    tracks = playlist["tracks"];
    songs = tracks["items"];
    total_tracks = tracks["total"]
    
#     # Iterate over all tracks in the playlist using offset to get beyond the first 100 tracks
    for i in range(100, total_tracks, 100):
        tracks = sp.user_playlist_tracks(username, playlistID, offset=i)
        songs += tracks["items"]
        
    #Putting Track ID, Track Name, Album, and artist name in matrix's

    track_ids = [] 
    track_names = []
    album = []
    artist =[]

    #Getting Track Data
    for i in range(0, len(songs)):
        if songs[i]['track']['id'] != None: # Removes the local tracks in your playlist if there is any
            track_ids.append(songs[i]['track']['id'])
            track_names.append(songs[i]['track']['name'])
            album.append(songs[i]['track']['album']['name'])
            artist.append(songs[i]['track']['artists'][0]['name'])

    features = []
    for i in range(0,len(songs)):
        audio_features = sp.audio_features(track_ids[i])
        for track in audio_features:

          if track is None:
            print(track)
            features.append({'danceability': 0, 'energy': 0, 'key': 0, 'loudness': 0, 'mode': 0, 'speechiness': 0, 'acousticness': 0, 'instrumentalness': 0, 'liveness': 0, 'valence': 0, 'tempo': 0, 'type': 'audio_features', 'id': '00000', 'uri': 'spotify:track:0', 'track_href': 'https://api.spotify.com/', 'analysis_url': 'https://api.spotify.com/', 'duration_ms': 0, 'time_signature': 0})
          else:
            features.append(track)

    #Creating our DataFrames
    df = pd.DataFrame(features, index = track_names) 

    #Adding a new column
    df['genre'] = genre #Country
    df['Song Title'] = track_names
    df['Artist'] = artist

    # Scale between 0 and 1
    df["key"] = (df["key"] / df["key"].max())
    df["tempo"] = (df["tempo"] / df["tempo"].max())
    df["loudness"] = (df["loudness"] / df["loudness"].min())
    
#     #calculate the IQR for instrumentalness and acousticness
    Q1 = df["instrumentalness"].quantile(0.25)
    Q3 = df["instrumentalness"].quantile(0.75)
    IQR = Q3 - Q1
    
    Q12 = df["acousticness"].quantile(0.25)
    Q32 = df["acousticness"].quantile(0.75)
    IQR2 = Q32 - Q12
    
#     #Remove rows with outliers
    df = df[~((df["instrumentalness"] < (df["instrumentalness"].quantile(0.25) - 1.5 * IQR)) | (df["instrumentalness"] > (df["instrumentalness"].quantile(0.75) + 1.5 * IQR)))]
    df = df[~((df["acousticness"] < (df["acousticness"].quantile(0.25) - 1.5 * IQR2)) | (df["acousticness"] > (df["acousticness"].quantile(0.75) + 1.5 * IQR2)))]
    
#     Q1 = df.quantile(0.25)
#     Q3 = df.quantile(0.75)
#     IQR = Q3 - Q1
#     df = df[~((df.sub(Q1, axis=1).div(IQR, axis=1)).lt(-1.5) | (df.sub(Q1, axis=1).div(IQR, axis=1)).gt(1.5)).any(axis=1)]
    return df

In [4]:
#Bringing in our Test Playlists
countryPlaylistID1 = 'https://open.spotify.com/playlist/37i9dQZF1DWZBCPUIUs2iR?si=8aa077376b294156' #Classic Country
countryPlaylistID2 = 'https://open.spotify.com/playlist/37i9dQZF1DX1lVhptIYRda?si=b23ff754227743cb' #Hot Country
countryPlaylistID3 = 'https://open.spotify.com/playlist/37i9dQZF1DWXdiK4WAVRUW?si=6e926046bed44a06' #2010's Country
countryPlaylistID4 = 'https://open.spotify.com/playlist/37i9dQZF1DXdxUH6sNtcDe?si=05b42d9d17234341' #2000's Country

country1 = input_playlist(countryPlaylistID1, 'country')
country2 = input_playlist(countryPlaylistID2, 'country')
country3 = input_playlist(countryPlaylistID3, 'country')
country4 = input_playlist(countryPlaylistID4, 'country')

In [16]:
rockPlaylistID1 ='https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=b4fc7abf24d943b7' #Classic Rock
rockPlaylistID2 ='https://open.spotify.com/playlist/37i9dQZF1DXcF6B6QPhFDv?si=0f691a7899ff4095' #Todays Rock
rockPlaylistID3 ='https://open.spotify.com/playlist/37i9dQZF1DXb3m918yXHxA?si=e633075567814062' #Yacht Rock
rockPlaylistID4 ='https://open.spotify.com/playlist/37i9dQZF1DX11ghcIxjcjE?si=b4f5ba0735d648a8' #Grunge Rock


rock1 = input_playlist(rockPlaylistID1, 'rock')
rock2 = input_playlist(rockPlaylistID2, 'rock')
rock3 = input_playlist(rockPlaylistID3, 'rock')
rock4 = input_playlist(rockPlaylistID4, 'rock')

In [6]:
rapPlaylistID1 ='https://open.spotify.com/playlist/0h9Gaqt2sNJ8M5aMV3h9BO?si=458a36ab9c32432c' #Rap
rapPlaylistID2 ='https://open.spotify.com/playlist/37i9dQZF1DXbkfWVLd8wE3?si=d5aba753953e42d4' 
rapPlaylistID3 ='https://open.spotify.com/playlist/4riovLwMCrY3q0Cd4e0Sqp?si=d21e717925fe49cb' 
rapPlaylistID4 ='https://open.spotify.com/playlist/37i9dQZF1DX0XUsuxWHRQd?si=6bb776c906c64e6c' 

rap1 = input_playlist(rapPlaylistID1, 'rap')
rap2 = input_playlist(rapPlaylistID2, 'rap')
rap3 = input_playlist(rapPlaylistID3, 'rap')
rap4 = input_playlist(rapPlaylistID4, 'rap')

In [7]:
rnbPlaylistID1 ='https://open.spotify.com/playlist/37i9dQZF1DX04mASjTsvf0?si=b73971c2510f4200' #R&B Classics
rnbPlaylistID2 ='https://open.spotify.com/playlist/37i9dQZF1DX4SBhb3fqCJd?si=80820f22c4d74275' #Are & Be
rnbPlaylistID3 ='https://open.spotify.com/playlist/37i9dQZF1DWSfMe9z89s9B?si=9b99e875d2ee487e' #Alternative RnB
rnbPlaylistID4 ='https://open.spotify.com/playlist/37i9dQZF1DWVGy1YP1ojM5?si=794cf8c95c634d6f' #Silky Sheets

rnb1 = input_playlist(rnbPlaylistID1, 'rnb')
rnb2 = input_playlist(rnbPlaylistID2, 'rnb')
rnb3 = input_playlist(rnbPlaylistID3, 'rnb')
rnb4 = input_playlist(rnbPlaylistID4, 'rnb')

In [9]:
popPlaylistID1 = 'https://open.spotify.com/playlist/7iUGicmg9AkkFhkLiYqYpA?si=042aa88085c24876' #Pop Classics
popPlaylistID2 ='https://open.spotify.com/playlist/37i9dQZF1DWUa8ZRTfalHk?si=7aaf3db78fcb4148' #Today's Pop
popPlaylistID3 ='https://open.spotify.com/playlist/37i9dQZF1DXa2PvUpywmrr?si=957b345e2f314dfe' #Party Pop
# popPlaylistID4 ='https://open.spotify.com/playlist/37i9dQZF1DWVGy1YP1ojM5?si=794cf8c95c634d6f' #Silky Sheets

pop1 = input_playlist(popPlaylistID1, 'pop')
pop2 = input_playlist(popPlaylistID2, 'pop')
pop3 = input_playlist(popPlaylistID3, 'pop')
# pop4 = input_playlist(popPlaylistID4, 'pop')

In [10]:
# #Bringing in our Test Playlists
classicalPlaylistID = 'https://open.spotify.com/playlist/27Zm1P410dPfedsdoO9fqm?si=104c7b4c8cd747f0'
classical = input_playlist(classicalPlaylistID,'classical')

In [11]:
# # #Bringing in our Test Playlists
# # countryPlaylistID1 = 'https://open.spotify.com/playlist/37i9dQZF1DWZBCPUIUs2iR?si=8aa077376b294156' #Classic Country
# # countryPlaylistID2 = 'https://open.spotify.com/playlist/37i9dQZF1DX1lVhptIYRda?si=b23ff754227743cb' #Hot Country
# # countryPlaylistID3 = 'https://open.spotify.com/playlist/37i9dQZF1DWXdiK4WAVRUW?si=6e926046bed44a06' #2010's Country
# # countryPlaylistID4 = 'https://open.spotify.com/playlist/37i9dQZF1DXdxUH6sNtcDe?si=05b42d9d17234341' #2000's Country


# # rockPlaylistID1 ='https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=b4fc7abf24d943b7' #Classic Rock
# # rockPlaylistID2 ='https://open.spotify.com/playlist/37i9dQZF1DXcF6B6QPhFDv?si=0f691a7899ff4095' #Todays Rock
# # rockPlaylistID3 ='https://open.spotify.com/playlist/37i9dQZF1DXb3m918yXHxA?si=e633075567814062' #Yacht Rock
# # rockPlaylistID4 ='https://open.spotify.com/playlist/37i9dQZF1DX11ghcIxjcjE?si=b4f5ba0735d648a8' #Grunge Rock

# # rapPlaylistID = 'https://open.spotify.com/playlist/4riovLwMCrY3q0Cd4e0Sqp?si=d21e717925fe49cb'
# # # rnbPlaylistID ='https://open.spotify.com/playlist/37i9dQZF1DX04mASjTsvf0?si=b73971c2510f4200'
# # popPlaylistID = 'https://open.spotify.com/playlist/7iUGicmg9AkkFhkLiYqYpA?si=042aa88085c24876'
# classicalPlaylistID = 'https://open.spotify.com/playlist/27Zm1P410dPfedsdoO9fqm?si=104c7b4c8cd747f0'

# # country = input_playlist(countryPlaylistID, 0)
# # rock = input_playlist(rockPlaylistID, 1)
# # rap = input_playlist(rapPlaylistID, 2)
# # rnb = input_playlist(rnbPlaylistID,3)
# # pop = input_playlist(popPlaylistID,4)
# # classical = input_playlist(classicalPlaylistID,5)


# # country1 = input_playlist(countryPlaylistID1, 'country')
# # country2 = input_playlist(countryPlaylistID2, 'country')
# # country3 = input_playlist(countryPlaylistID3, 'country')
# # country4 = input_playlist(countryPlaylistID4, 'country')


# # rock1 = input_playlist(rockPlaylistID1, 'rock')
# # rock2 = input_playlist(rockPlaylistID2, 'rock')
# # rock3 = input_playlist(rockPlaylistID3, 'rock')
# # rock4 = input_playlist(rockPlaylistID4, 'rock')

# # rap = input_playlist(rapPlaylistID, 'rap')
# # rnb = input_playlist(rnbPlaylistID,'rnb')
# # pop = input_playlist(popPlaylistID,'pop')
# classical = input_playlist(classicalPlaylistID,'classical')


In [17]:
#Putting all of our playlists together
country = pd.concat([country1,country2,country3,country4[:1861]], ignore_index = True)
rock = pd.concat([rock1,rock2,rock3,rock4[:1861]], ignore_index = True)
rap = pd.concat([rap1,rap2,rap3,rap4[:1861]], ignore_index = True)
rnb = pd.concat([rnb1,rnb2,rnb3,rnb4[:1861]], ignore_index = True)
pop = pd.concat([pop1,pop2,pop3[:1861]], ignore_index = True)


In [20]:
#Putting all the data together
full_data = pd.concat([rock,country,rnb,pop,classical,rap[:1861]], ignore_index = True)

full_data_random = full_data.sample(frac=1)
# full_data_random

# #Just getting the numerical values, so no ID's, Artists, ect...
num_data_random = full_data_random[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature','genre']]
num_data_random

#Sending our data to a csv to save computing power and time. All the data is always the same. 
full_data.to_csv('full_data.csv', index = False)
full_data_random.to_csv('full_data_random.csv', index=False)
num_data_random.to_csv('num_data_random.csv', index=False)

In [21]:
num_data_random

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genre
1845,0.977,0.3790,0.090909,1.000000,1,0.2550,0.0190,0.000000,0.0900,0.5060,0.724349,4,rap
724,0.765,0.8210,0.363636,0.346206,0,0.1830,0.0918,0.000000,0.2170,0.9620,0.792752,4,rnb
175,0.381,0.8100,0.090909,0.348180,0,0.0472,0.0216,0.000000,0.0866,0.1470,0.484902,4,rock
1829,0.752,0.6870,0.090909,0.371454,0,0.0950,0.0171,0.000000,0.1270,0.2660,0.867055,4,rap
1177,0.779,0.7870,0.909091,0.397177,0,0.1080,0.0524,0.000000,0.1400,0.7080,0.574239,4,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,0.842,0.7030,0.727273,0.469884,1,0.2060,0.2290,0.000000,0.1120,0.8110,0.463494,4,rnb
1417,0.331,0.0118,0.636364,0.779734,1,0.0370,0.9940,0.905000,0.0762,0.0577,0.338504,4,classical
338,0.600,0.5600,0.727273,0.307734,1,0.0325,0.4790,0.000000,0.1110,0.4170,0.683618,4,country
1676,0.660,0.7750,0.181818,0.315425,1,0.1720,0.0553,0.000000,0.1290,0.6350,0.777348,4,rap
